In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/raw/fake_real_job_postings_3000x25.csv")
df.head()


,job_id,job_title,job_description,requirements,benefits,company_name,company_profile,industry,employment_type,location,...,application_deadline,contact_email,company_website,has_logo,num_open_positions,job_function,telecommuting,fraud_reason,text_length,is_fake
0,1,Software Engineer,We are looking for responsibilities fast-paced...,Candidates should have dynamic team skills fas...,We offer required skills fast-paced skills req...,Company_543,Our company growth fast-paced responsibilities...,Marketing,Contract,"Toronto, Canada",...,2024-09-16,hr312@company.com,https://www.company.com,0,3,Management,0,NaN,89,0
1,2,Content Writer,We are looking for required support experience...,Candidates should have required team fast-pace...,We offer fast-paced dynamic dynamic strategy g...,Company_192,Our company fast-paced opportunity innovation ...,Finance,Full-time,"Toronto, Canada",...,2024-10-18,hr127@company.com,https://www.company.com,0,10,Development,1,NaN,89,0
2,3,Customer Support Specialist,We are looking for dynamic required fast-paced...,Candidates should have preferred knowledge opp...,We offer skills experience required growth res...,NaN,We are global innovation growth skills knowled...,Healthcare,Internship,Remote,...,2024-01-13,job92@gmail.com,NaN,0,6,Support,0,Suspicious email,69,1
3,4,Data Analyst,We are looking for collaboration skills suppor...,Candidates should have innovation team require...,We offer strategy strategy dynamic support opp...,Company_95,Our company fast-paced support team strategy i...,Healthcare,Part-time,"Berlin, Germany",...,2024-02-09,hr366@company.com,https://www.company.com,1,4,Management,1,NaN,89,0
4,5,Graphic Designer,We are looking for team growth growth fast-pac...,Candidates should have experience preferred kn...,We offer opportunity skills responsibilities c...,NaN,We are global experience skills preferred fast...,Retail,Part-time,"London, UK",...,2024-08-26,job359@gmail.com,NaN,0,5,Management,0,No salary info,69,1


In [2]:
df.columns

Index(['job_id', 'job_title', 'job_description', 'requirements', 'benefits',
       'company_name', 'company_profile', 'industry', 'employment_type',
       'location', 'salary_range', 'required_experience_years',
       'education_level', 'department', 'posting_date', 'application_deadline',
       'contact_email', 'company_website', 'has_logo', 'num_open_positions',
       'job_function', 'telecommuting', 'fraud_reason', 'text_length',
       'is_fake'],
      dtype='object')

In [3]:
df["company_profile_present"] = df["company_profile"].notna().astype(int)
df["company_website_present"] = df["company_website"].notna().astype(int)
df["contact_email_present"] = df["contact_email"].notna().astype(int)
df["salary_range_present"] = df["salary_range"].notna().astype(int)


In [4]:
binary_features = df[
    ["has_logo", "telecommuting"]
].fillna(0)


In [11]:
df[["job_id","has_logo", "telecommuting","company_profile_present", "company_website_present", "contact_email_present", "salary_range_present"]]

,job_id,has_logo,telecommuting,company_profile_present,company_website_present,contact_email_present,salary_range_present
0,1,0,0,1,1,1,1
1,2,0,1,1,1,1,1
2,3,0,0,1,0,1,1
3,4,1,1,1,1,1,1
4,5,0,0,1,0,1,1
...,...,...,...,...,...,...,...
2995,2996,0,1,1,1,1,1
2996,2997,0,0,1,0,1,1
2997,2998,0,1,1,0,1,1
2998,2999,0,1,1,0,1,1


In [12]:
df["num_open_positions"] = df["num_open_positions"].fillna(0)
df["required_experience_years"] = df["required_experience_years"].fillna(0)
df["text_length"] = df["text_length"].fillna(0)


In [13]:
df["employment_type"] = df["employment_type"].fillna("Unknown")

employment_type_encoded = pd.get_dummies(
    df["employment_type"],
    prefix="employment_type"
)


In [14]:
X_aux = pd.concat(
    [
        df[
            [
                "company_profile_present",
                "company_website_present",
                "contact_email_present",
                "salary_range_present",
                "num_open_positions",
                "required_experience_years",
                "text_length"
            ]
        ],
        binary_features,
        employment_type_encoded
    ],
    axis=1
)


In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_aux_scaled = scaler.fit_transform(X_aux)


In [ ]:
np.save("../data/processed/X_aux.npy", X_aux_scaled)


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import re

    

Libraries imported successfully ✅


In [19]:
bilstm_model = tf.keras.models.load_model("../models/bilstm_model.keras")
print("Bi-LSTM model loaded successfully ✅")


Bi-LSTM model loaded successfully ✅


c:\Users\heman\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 11 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
feature_extractor = tf.keras.Model(
    inputs=bilstm_model.input,
    outputs=bilstm_model.get_layer("bilstm_layer").output
)

print("Feature extractor created from Bi-LSTM layer ")


Feature extractor created from Bi-LSTM layer ✅


In [22]:
with open("../models/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

print("Tokenizer loaded successfully ")


Tokenizer loaded successfully 


In [24]:
df = pd.read_csv("../data/raw/fake_real_job_postings_3000x25.csv")
print("Dataset loaded successfully ")
print("Total records:", len(df))


Dataset loaded successfully 
Total records: 3000


In [25]:
text_columns = ["job_title", "job_description", "requirements", "benefits"]
df[text_columns] = df[text_columns].fillna("")

df["merged_text"] = (
    df["job_title"] + " " +
    df["job_description"] + " " +
    df["requirements"] + " " +
    df["benefits"]
)

print("Text columns merged successfully ")
print("Sample merged text:\n")
print(df["merged_text"].iloc[0][:300], "...")


Text columns merged successfully 
Sample merged text:

Software Engineer We are looking for responsibilities fast-paced fast-paced growth development team innovation skills experience team dynamic fast-paced experience dynamic innovation fast-paced strategy responsibilities experience opportunity innovation required responsibilities growth dynamic requi ...


In [26]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_text"] = df["merged_text"].apply(clean_text)

print("Text cleaned successfully")
print("Sample cleaned text:\n")
print(df["clean_text"].iloc[0][:300], "...")


Text cleaned successfully
Sample cleaned text:

software engineer we are looking for responsibilities fast paced fast paced growth development team innovation skills experience team dynamic fast paced experience dynamic innovation fast paced strategy responsibilities experience opportunity innovation required responsibilities growth dynamic requi ...


In [27]:
MAX_LEN = 300

sequences = tokenizer.texts_to_sequences(df["clean_text"])
X_text = tf.keras.preprocessing.sequence.pad_sequences(
    sequences,
    maxlen=MAX_LEN,
    padding="post",
    truncating="post"
)

print("Text tokenized and padded successfully ")
print("Text tensor shape:", X_text.shape)


Text tokenized and padded successfully 
Text tensor shape: (3000, 300)


In [29]:
X_bilstm_features = feature_extractor.predict(X_text, batch_size=32)

print("Bi-LSTM feature extraction completed ✅")
print("Feature vector shape:", X_bilstm_features.shape)


94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 107ms/step
Bi-LSTM feature extraction completed ✅
Feature vector shape: (3000, 256)


In [30]:
np.save("../data/processed/X_bilstm_features.npy", X_bilstm_features)

print("Bi-LSTM feature vectors saved successfully ")


Bi-LSTM feature vectors saved successfully 


In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

print("Libraries imported successfully ✅")


Libraries imported successfully ✅


In [35]:
X_bilstm = np.load("../data/processed/X_bilstm_features.npy")
print("Bi-LSTM feature vectors loaded ")
print("Bi-LSTM feature shape:", X_bilstm.shape)
X_aux = np.load("../data/processed/X_aux.npy")
print("Auxiliary features loaded ")
print("Auxiliary feature shape:", X_aux.shape)
X_final = np.concatenate([X_bilstm, X_aux], axis=1)

print("Bi-LSTM + Auxiliary features combined successfully ")
print("Final feature vector shape:", X_final.shape)




Bi-LSTM feature vectors loaded 
Bi-LSTM feature shape: (3000, 256)
Auxiliary features loaded 
Auxiliary feature shape: (3000, 14)
Bi-LSTM + Auxiliary features combined successfully 
Final feature vector shape: (3000, 270)


In [36]:
y = pd.read_csv("../data/raw/fake_real_job_postings_3000x25.csv")["is_fake"].values

print("Labels loaded successfully ")
print("Total labels:", len(y))


Labels loaded successfully 
Total labels: 3000


In [37]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X_final, y, test_size=0.30, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print("Dataset split completed ")
print("Training samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])
print("Testing samples:", X_test.shape[0])


Dataset split completed 
Training samples: 2100
Validation samples: 450
Testing samples: 450


In [38]:
mlp_model = Sequential([
    Dense(128, activation="relu", input_shape=(X_final.shape[1],)),
    Dropout(0.4),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

print("MLP model architecture created ")


MLP model architecture created 


c:\Users\heman\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
mlp_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

mlp_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        34,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,009 (168.00 KB)

 Trainable params: 43,009 (168.00 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)
print("MLP training started...............! ")


history = mlp_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stop]
)

print("MLP training completed successfully ")


MLP training started...............! 
Epoch 1/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9722 - loss: 0.1154 - val_accuracy: 1.0000 - val_loss: 2.4468e-05
Epoch 2/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 2.9094e-04 - val_accuracy: 1.0000 - val_loss: 1.0508e-05
Epoch 3/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 5.1532e-04 - val_accuracy: 1.0000 - val_loss: 3.8092e-06
Epoch 4/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.2931e-04 - val_accuracy: 1.0000 - val_loss: 1.9752e-06
Epoch 5/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 5.9018e-05 - val_accuracy: 1.0000 - val_loss: 1.2206e-06
Epoch 6/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 8.6949e-05 - val_accuracy: 1.0000 - val_loss: 7.2364e-07
Epoch 7/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 5.5115e-05 - val_accuracy: 1.0000 - val_loss: 4.2727e-07
Epoch 8/20
66/66 ━━━━━━━━━━

In [41]:
test_loss, test_accuracy = mlp_model.evaluate(X_test, y_test)

print("Final MLP Test Accuracy:", test_accuracy)
print("Final MLP Test Loss:", test_loss)


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 2.2675e-08 
Final MLP Test Accuracy: 1.0
Final MLP Test Loss: 2.2090238971372855e-08


In [43]:
mlp_model.save("../models/mlp_model.keras")
print("MLP model saved successfully ✅")


MLP model saved successfully ✅


In [3]:
import numpy as np
X_aux = np.load("../data/processed/X_aux.npy")
print(X_aux.shape)


(3000, 14)
